<a href="https://colab.research.google.com/github/ShikharGhimire/Diabetic-Retinopathy/blob/main/SimpleUnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Segmenting retina blood vessels using UNET


In [ ]:
#Accesing the google drive
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
#Importing libraries
import os
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import HorizontalFlip, VerticalFlip, Rotate

In [ ]:
if __name__=='__main__':
  np.random.seed(42)

In [ ]:
#Loading the dataset
def load_path(path):
  train_x = sorted(glob(os.path.join(folder_path,'train','training','images','*.tif'))) #Image folder
  train_y = sorted(glob(os.path.join(folder_path,'train','markup','*.gif'))) #Mask images y
  #For testing folder
  test_x = sorted(glob(os.path.join(folder_path,'test','images','*.tif'))) #Image folder
  test_y = sorted(glob(os.path.join(folder_path,'test','mask','*.gif'))) #Mask images y
  
  return (train_x,train_y), (test_x,test_y)

In [ ]:
#Loading the dataset
folder_path = '/content/gdrive/My Drive/DBR/DRIVE'
(train_x,train_y),(test_x,test_y) = load_path(folder_path)

In [ ]:
#Let's check the length of the data
print('Length of training data is {}-{}'.format(len(train_x),len(train_y)))
print('Length of testing data is {}-{}'.format(len(test_x),len(test_y)))

Length of training data is 20-20
Length of testing data is 20-20


In [ ]:
#Let's check the dimension of the image
training_images = []
mask_images = []
for i in train_x:
  training_images.append(i)
for j in train_y:
  mask_images.append(j)

In [ ]:
image_size = set([cv2.imread(x).shape for x in training_images])
print('All the image sizes are',image_size)

#All the image size is that of 584x563 with 3 channels

#Let's look at mask dimensions
mask_size = set([imageio.mimread(x)[0].shape for x in mask_images])
print('All the mask sizes are',mask_size)

#All the mask size is that of 584x565

All the image sizes are {(584, 565, 3)}
All the mask sizes are {(584, 565)}


In [ ]:
# #Data Augmentation for the retinal eye images
# def create_dir(path):
#   if not os.path.exists(path):
#     os.makedirs(path)

# #Creating directories to save the augmented data
# create_dir('/content/gdrive/My Drive/DBR/DRIVE/aug_train/images')
# create_dir('/content/gdrive/My Drive/DBR/DRIVE/aug_train/mask')
# create_dir('/content/gdrive/My Drive/DBR/DRIVE/aug_test/image')
# create_dir('/content/gdrive/My Drive/DBR/DRIVE/aug_test/mask')

In [ ]:
#Let's create some augmented image
def augmented_data(images,masks,save_path,augment = True):
  size = (560,560)
  for idx, (x,y) in tqdm(enumerate(zip(images,masks)),total = len(images)):
    image_name = x.split('/')[-1].split('.')[0] #Extracting the name of the file
    #Reading the image and the mask
    x = cv2.imread(x,cv2.IMREAD_COLOR)
    y = imageio.mimread(y)[0]
    
    if augment == True: #If we want to do the augmentation
      #Horizontal flipping
      aug = HorizontalFlip(p=1.0)
      augmented = aug(image = x, mask = y)
      x1 = augmented['image']
      y1 = augmented['mask']

      #Vertical Flipping
      aug = VerticalFlip(p = 1.0)
      augmented = aug(image=x,mask = y)
      x2 = augmented['image']
      y2 = augmented['mask']

      #Rotating
      aug = Rotate(limit = 45, p = 1.0)
      augmented = aug(image=x,mask = y)
      x3 = augmented['image']
      y3 = augmented['mask']

      X = [x,x1,x2,x3]
      Y = [y,y1,y2,y3]

    else:
      X = [x]
      Y = [y]

    index = 0
    for i,m in zip(X,Y):
      i = cv2.resize(i,size)
      m = cv2.resize(m,size)

      #Creating temporary name to store all the augmented data
      tmp_image_name = f"{image_name}_{index}.png"
      tmp_mask_name = f"{image_name}_{index}.png"

      image_path = os.path.join(save_path,"images",tmp_image_name)
      mask_path =  os.path.join(save_path,"mask",tmp_mask_name)

      cv2.imwrite(image_path,i)
      cv2.imwrite(mask_path,m)

      index = index+1


In [ ]:
augmented_data(train_x, train_y,'/content/gdrive/My Drive/DBR/DRIVE/aug_train/', augment = True) #For training data
augmented_data(train_x,train_y,'/content/gdrive/My Drive/DBR/DRIVE/aug_test/',augment = False) #For testing data as we do not need to do any augmentation on it we put augment as false

100%|██████████| 20/20 [00:00<00:00, 26.33it/s]


### Pytorch Unet architecture


In [ ]:
import torch
import torch.nn as nn

In [ ]:
class conv_block(nn.Module):
  def __init__(self,in_c,out_c):
    super().__init__()
    #First convolution layer
    self.conv1 = nn.Conv2d(in_c, out_c, kernel_size = 3, padding =1)

  def forward(self,inputs):
    x = self.conv1(inputs)
    print(x)

if __name__ == "__main__":
  x = torch.randn((2,32,128,128)) #batch size,number_of_channels,height,width 
  b = conv_block(32,64) #Calling the conv block class
  b(x)
